In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib as plt
import rasterio
import fiona

In [3]:
def rsei_calculator(ndvi, wetness, ndbsi, lst, year):
    #Minimum Maximum Value
    max_ndvi = ndvi.max()
    min_ndvi = ndvi.min()
    max_wet = wetness.max()
    min_wet = wetness.min()
    max_ndbsi = ndbsi.max()
    min_ndbsi = ndbsi.min()
    max_lst = lst.max()
    min_lst = lst.min()
    #Normalization
    norm_ndvi = (ndvi - min_ndvi)/(max_ndvi-min_ndvi)
    norm_wetness = (wetness-min_wet)/(max_wet-min_wet)
    norm_ndbsi = (max_ndbsi - ndbsi)/(max_ndbsi-min_ndbsi)
    norm_lst = (max_lst-lst)/(max_lst-min_lst)
    #Probability
    prob_ndvi = norm_ndvi/norm_ndvi.sum()
    prob_wetness = norm_wetness/norm_wetness.sum()
    prob_ndbsi = norm_ndbsi/norm_ndbsi.sum()
    prob_lst = norm_lst/norm_lst.sum()
    #Uniqe Value
    ndvi_unqe = prob_ndvi.nunique()
    wet_unqe = prob_wetness.nunique()
    ndbsi_unqe = prob_ndbsi.nunique()
    lst_unqe = prob_lst.nunique()
    #Constant Value
    k_ndvi = 1/np.log(ndvi_unqe)
    k_wet = 1/np.log(wet_unqe)
    k_ndbsi = 1/np.log(ndbsi_unqe)
    k_lst = 1/np.log(lst_unqe)
    #Entropy Calculation
    en_ndvi = -k_ndvi * (prob_ndvi*prob_ndvi.sum())
    en_wetness = -k_wet * (prob_wetness*prob_wetness.sum())
    en_ndbsi = -k_ndbsi * (prob_ndbsi*prob_ndbsi.sum())
    en_lst = -k_lst * (prob_lst*prob_lst.sum())
    #Weighted Entropy
    df[f'w_en_ndvi_{year}'] = (1-en_ndvi)/(4-en_ndvi.sum())
    df[f'w_en_wetness_{year}'] = (1-en_wetness)/(4-en_wetness.sum())
    df[f'w_en_ndbsi_{year}'] = (1-en_ndbsi)/(4-en_ndbsi.sum())
    df[f'w_en_lst_{year}'] = (1-en_lst)/(4-en_lst.sum())

    w_en_ndvi = (1-en_ndvi)/(4-en_ndvi.sum())
    w_en_wetness = (1-en_wetness)/(4-en_wetness.sum())
    w_en_ndbsi = (1-en_ndbsi)/(4-en_ndbsi.sum())
    w_en_lst = (1-en_lst)/(4-en_lst.sum())
    #RSEI
    df[f'rsei_{year}'] = (norm_ndvi*w_en_ndvi) + (norm_wetness*w_en_wetness) + (norm_ndbsi*w_en_ndbsi) + (norm_lst*w_en_lst)

In [5]:
filePath = r"C:\Users\USER\OneDrive - The University of Memphis\Khulna University\BlueGreenSpace\EcologicalIndexGIS\Coding\CCC_update.csv"

In [7]:
df = pd.read_csv(filePath)
df

,pointid,grid_code,Lat,Long,wetness_24_CCC,wetness_20_CCC,ndvi_24_CCC,ndvi_20_CCC,ndbsi_24_CCC,ndbsi_20_CCC,lst_24_CCC,lst_20_CCC,ndvi_16_CCC,wetness_16_CCC,ndbsi_16_CCC,lst_16_CCC
0,375100,0.097382,22.284103,91.755495,0.097382,0.103108,-0.315530,-0.333333,-1.577689,-1.739226,24.057121,19.931996,-0.109455,0.125311,-1.699626,20.548001
1,375101,0.095834,22.284103,91.755765,0.095834,0.105078,-0.335390,-0.301850,-1.421008,-1.829646,24.109618,19.940285,-0.107610,0.125555,-1.690545,20.535249
2,375102,0.091839,22.284103,91.756034,0.091839,0.109309,-0.402614,-0.245320,-1.011209,-2.250006,24.147370,19.956297,-0.124121,0.123792,-1.574498,20.539362
3,375103,0.091868,22.284103,91.756304,0.091868,0.113714,-0.403792,-0.187133,-1.016581,-2.625565,24.169326,19.968449,-0.137976,0.122738,-1.518534,20.560643
4,375104,0.093551,22.284103,91.756573,0.093551,0.113207,-0.379445,-0.193228,-1.083550,-2.640080,24.192507,19.981709,-0.134075,0.123148,-1.532340,20.585419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241487,108555,0.084735,22.413999,91.885661,0.084735,0.077586,-0.277156,-0.351399,-0.755090,-0.842305,25.855145,21.168719,-0.087347,0.090804,-0.693800,20.549345
241488,108556,0.087608,22.413999,91.885931,0.087608,0.078240,-0.290166,-0.341877,-1.030044,-0.835381,25.329644,21.078288,-0.098012,0.091841,-0.761193,20.492766
241489,108557,0.087608,22.413999,91.886200,0.087608,0.078240,-0.290166,-0.341877,-1.030044,-0.835381,25.329644,21.078288,-0.098012,0.091841,-0.761193,20.492766
241490,108558,0.088928,22.413999,91.886470,0.088928,0.078919,-0.299395,-0.346815,-1.236112,-0.872059,24.919990,21.046026,-0.102430,0.092465,-0.776248,20.450511


In [9]:
# Applying the method
check_nan = df.isnull().values.any()

# printing the result
print(check_nan)

False


In [11]:
ndvi = df['ndvi_24_CCC']
wetness = df['wetness_24_CCC']
ndbsi = df['ndbsi_24_CCC']
lst = df['lst_24_CCC']

In [13]:
rsei_calculator(ndvi, wetness, ndbsi, lst, 24)

In [17]:
ndvi = df['ndvi_20_CCC']
wetness = df['wetness_20_CCC']
ndbsi = df['ndbsi_20_CCC']
lst = df['lst_20_CCC']

In [15]:
rsei_calculator(ndvi, wetness, ndbsi, lst, 20)

In [17]:
ndvi = df['ndvi_16_CCC']
wetness = df['wetness_16_CCC']
ndbsi = df['ndbsi_16_CCC']
lst = df['lst_16_CCC']

In [19]:
rsei_calculator(ndvi, wetness, ndbsi, lst, 16)
df

,pointid,grid_code,Lat,Long,wetness_24_CCC,wetness_20_CCC,ndvi_24_CCC,ndvi_20_CCC,ndbsi_24_CCC,ndbsi_20_CCC,...,w_en_ndvi_20,w_en_wetness_20,w_en_ndbsi_20,w_en_lst_20,rsei_20,w_en_ndvi_16,w_en_wetness_16,w_en_ndbsi_16,w_en_lst_16,rsei_16
0,375100,0.097382,22.284103,91.755495,0.097382,0.103108,-0.315530,-0.333333,-1.577689,-1.739226,...,0.245017,0.245025,0.245019,0.245025,0.576177,0.245017,0.245025,0.24502,0.245025,0.703940
1,375101,0.095834,22.284103,91.755765,0.095834,0.105078,-0.335390,-0.301850,-1.421008,-1.829646,...,0.245017,0.245025,0.245019,0.245025,0.567565,0.245017,0.245025,0.24502,0.245025,0.704049
2,375102,0.091839,22.284103,91.756034,0.091839,0.109309,-0.402614,-0.245320,-1.011209,-2.250006,...,0.245017,0.245025,0.245019,0.245025,0.543593,0.245017,0.245025,0.24502,0.245025,0.688573
3,375103,0.091868,22.284103,91.756304,0.091868,0.113714,-0.403792,-0.187133,-1.016581,-2.625565,...,0.245017,0.245025,0.245019,0.245025,0.543235,0.245017,0.245025,0.24502,0.245025,0.678959
4,375104,0.093551,22.284103,91.756573,0.093551,0.113207,-0.379445,-0.193228,-1.083550,-2.640080,...,0.245017,0.245025,0.245019,0.245025,0.549479,0.245017,0.245025,0.24502,0.245025,0.680808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241487,108555,0.084735,22.413999,91.885661,0.084735,0.077586,-0.277156,-0.351399,-0.755090,-0.842305,...,0.245017,0.245025,0.245019,0.245025,0.535548,0.245017,0.245025,0.24502,0.245025,0.608134
241488,108556,0.087608,22.413999,91.885931,0.087608,0.078240,-0.290166,-0.341877,-1.030044,-0.835381,...,0.245017,0.245025,0.245019,0.245025,0.547721,0.245017,0.245025,0.24502,0.245025,0.612878
241489,108557,0.087608,22.413999,91.886200,0.087608,0.078240,-0.290166,-0.341877,-1.030044,-0.835381,...,0.245017,0.245025,0.245019,0.245025,0.547721,0.245017,0.245025,0.24502,0.245025,0.612878
241490,108558,0.088928,22.413999,91.886470,0.088928,0.078919,-0.299395,-0.346815,-1.236112,-0.872059,...,0.245017,0.245025,0.245019,0.245025,0.556906,0.245017,0.245025,0.24502,0.245025,0.614241


In [21]:
df.to_csv('CCC_rmsi_update.csv', header=True, index=False)